# Context Manager

`Context Managers` in `Python` allows us to manage the resources without having to explicitly manage them. Say you want to perform some operation and now, your biggest worry is what will happen if for some reason, file was not closed. 

With context managers you don't have to worry about them. It will take care of it for you transparently. 

## Creating a Context Manager class

## Creating a Context Manager using contextlib

## contextlib.closing(thing)

## contextlib.suppress(*exceptions)

## contextlib.redirect_stdout / redirect_stderr

## ExitStack

## Reentrant Context Managers

## Reference

- https://www.python.org/dev/peps/pep-0343/
    